# Deep Learning with Python
# 5.2.2 - Base ConvNet

## Instantiating Model

In [1]:
from tensorflow.keras import models, layers

In [2]:
model = models.Sequential()

# First block
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# Second block
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Third block
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Final block - filters have stopped increasing
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten via dot product with convolutional kernel
model.add(layers.Flatten())

# Densely connected classifier begins
model.add(layers.Dense(512, activation='relu'))

# Binary classification so ouput layer is single unit with sigmoid activation
# So that it outputs a probability between 0 and 1 of the input belonging to postive class
# Positive class is arbitrarily decided in this case 
model.add(layers.Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


## Model Summary
- Lotsa parameters

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       147584    
__________

## Compiling

In [4]:
# Defining a custom optimizer so we can specify a non-default learning rate
from tensorflow.keras import optimizers

# Training the model
model.compile(loss='binary_crossentropy', 
             optimizer=optimizers.RMSprop(lr=1e-4), 
             metrics=['acc'])

## Data Preprocessing
1. Read the picture files.
2. Decode the JPEG content into RGB grids of pixels.
3. Convert these into floating-point tensors.
4. Rescale the pixel vlaues (between 0 and 255) to the [0, 1] interval so that the neural network can deal with small input values.

### `ImageDataGenerator` - Read Data from Directory

An `ImageDataGenerator` object is capable of automatically converting image files on disk into batches of preprocessed tensors. It is packaged as part of `keras`.

I think the data generator uses the names of all the files it processes in the target directories to decide how many classes are there.

In [5]:
# Importing directory manipulation libary - allows Python to talk directly to OS
import os

In [6]:
# Defining paths for the training and test set directories
base_dir = '/Users/saads/OneDrive/Desktop/DL-Python/chapter-5/ch-5-repeat/cats_dogs_small'
train_dir  = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
validation_dir = os.path.join(base_dir, 'validation')

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
# Setting up an ImageDataGenerator to read files from both training and test sets

# Both image data generators will rescale their respective target files by 255
train_datagen = ImageDataGenerator(rescale=1./255)  
test_datagen = ImageDataGenerator(rescale=1./255)

# Specifying source of files for the train data generator along with 
# transformations specific to the training set images
train_generator = train_datagen.flow_from_directory(
    train_dir,                 # Target directory - where the data for this gen will come from
    target_size=(150, 150),    # Resizes all images to 150 x 150 
    batch_size=20,             # Image tensors generated in batches of 20 samples
    class_mode='binary')       # Because we will be using binary crossentropy

Found 2000 images belonging to 2 classes.


In [9]:
# Setting up a similar ImageDataGenerator to read files from the validation set
validation_generator = test_datagen.flow_from_directory(
    validation_dir, 
    target_size=(150, 150), 
    batch_size=20, 
    class_mode='binary')

Found 1000 images belonging to 2 classes.


The data generator will continuously iterate over the target directory, and will batches of 20 samples where each sample is an image tensor of size (150, 150, 3). Therefore, we also need to specify when the data generator has to stop iterating over the target directory i.e. stop converting images from the target directory into tensors.

In [10]:
for data_batch, labels_batch in train_generator:
    print('Data batch shape: ', data_batch.shape)
    print('Labels batch shape: ', labels_batch.shape)
    break

Data batch shape:  (20, 150, 150, 3)
Labels batch shape:  (20,)


## Fitting the Model to the Data
When we use a batch of tensors to train a model, we use the `fit` method. When the tensor data will be provided by a `generator` object that iterates over a target directory containing to transform some kind of input i.e. images into a tensor, we use the `fit_generator` method.

`fit_generator` must also be provided an argument that specifies how many times the generator must iterate over the directory to create training data - the equivalent of specifying when to `break` the loop that uses the generator. 

`steps_per_epoch` defines the number of batches of input tensors that will be drawn for each epoch. In this case, we have a total of 2000 training images, and each batch will draw 20 samples, so we need to draw 100 batches of samples. This is equivalent to running 100 gradient descent steps - one for each batch of samples drawn - before the next epoch begins.

Because the validation data is also provided by an `ImageGenerator` object, we must specify the number of steps for which the model will run for the validation set.

In this case, the validation data as 1000 samples. So we must generate 50 batches of 20 samples to iterate over the entire validation set.

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100, 
    epochs=30, 
    validation_data=validation_generator, 
    validation_steps=50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
100/100 [==============================] - 213s 2s/step - loss: 0.6907 - acc: 0.5315 - val_loss: 0.6670 - val_acc: 0.6140
Epoch 2/30
 41/100 [===========>..................] - ETA: 1:39 - loss: 0.6690 - acc: 0.5841

## Save the Model

In [17]:
model.save('cats_and_dogs_small.h5')

## Visualizing Performance

In [18]:
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

NameError: name 'history' is not defined

In [ ]:
# Accuracy Plot
plt.figure(); plt.plot(epochs, acc, 'bo', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b-', label='Validation Accuracy')
plt.xlabel('Epochs'); plt.ylabel('Accuracy'); plt.grid(True);
plt.legend(); plt.title('Original Model - Accuracy'; plt.show();

In [ ]:
# Loss Plot
plt.figure(); plt.plot(epochs, loss, 'bo', label='Training Loss');
plt.plot(epochs, val_loss, 'b-', label='Validation Loss')
plt.xlabel('Epochs'); plt.ylabel('Loss'); plt.grid(True);
plt.legend(); plt.title('Original Model - Loss'); plt.show();